In [ ]:
%%capture
! pip install transformers datasets evaluate
! pip install accelerate
! pip install --upgrade accelerate
! pip install huggingface_hub
! pip install wandb

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import wandb
wandb.login()

%env WANDB_PROJECT=distilBERT_finetuning
%env WANDB_LOG_MODEL=true

wandb: Currently logged in as: christian-159. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=distilBERT_finetuning
env: WANDB_LOG_MODEL=true


In [ ]:
from datasets import load_dataset, Dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import accelerate
from transformers import DataCollatorWithPadding

# Load data
labels = ClassLabel(num_classes=3, names=["negative", "neutral", "positive"])
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}


# Load dataset
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/masterProject/av_train.csv")
dataset = dataset.rename_column("finBERT", "label")
dataset = dataset.rename_column("summary", "text")

# Split into 80% training and 20% validation
dataset = dataset["train"].train_test_split(train_size=0.8)

# Tokenize dataset using distilbert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    examples["label"] = labels.str2int(examples["label"])
    return tokenizer(examples["text"], padding=True, truncation=True)


tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)

# Convert to PyTorch tensors for faster training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
def model_init():
  model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", id2label=id2label, label2id=label2id, num_labels=3)
  return model

In [ ]:
sweep_config = {
    'method': 'grid',
    'name': 'distilBERT_finetuning',
    'metric': {
        'goal': 'maximize',
        'name': 'eval/f1'
        },
    'parameters': {
        'per_device_train_batch_size': {
            'values': [32, 64, 128]
            },
        'learning_rate': {
            'values': [1e-5, 3e-5, 5e-5]
        },
        'weight_decay': {
            'values': [0.0, 0.25]
        }
     },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 5
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='distilBERT_finetuning')

Create sweep with ID: uahb15q7
Sweep URL: https://wandb.ai/christian-159/distilBERT_finetuning/sweeps/uahb15q7


In [1]:
def compute_metrics(eval_preds):
  metrics = dict()

  accuracy_metric = evaluate.load("accuracy")
  f1_metric = evaluate.load("f1")


  logits = eval_preds.predictions
  labels = eval_preds.label_ids
  preds = np.argmax(logits, axis=-1)

  metrics.update(accuracy_metric.compute(predictions=preds, references=labels))
  metrics.update(f1_metric.compute(predictions=preds, references=labels, average='weighted'))

  return metrics



def train(config=None):
  with wandb.init(config=config):
    # set sweep configuration
    config = wandb.config


    # set training arguments
    training_args = TrainingArguments(
        output_dir="distilBERT_finetuning",
        learning_rate=config.learning_rate,
        per_device_train_batch_size=config.per_device_train_batch_size,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        weight_decay=config.weight_decay,
        evaluation_strategy="steps",
        report_to="wandb",
        eval_steps=25,
        max_steps = 200,
        save_steps = 0,
        load_best_model_at_end=True,
        logging_steps=1
    )


    # define training loop
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )


    # start training loop
    trainer.train()

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: xh8a03lp with config:
wandb: 	learning_rate: 1e-05
wandb: 	per_device_train_batch_size: 32
wandb: 	weight_decay: 0


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  

Step,Training Loss,Validation Loss,Accuracy,F1
25,1.056800,1.052234,0.423750,0.258076
50,0.939300,0.952162,0.580000,0.503611
75,0.812200,0.809131,0.717500,0.690900
100,0.757500,0.710850,0.740000,0.726773
125,0.611700,0.647887,0.753750,0.741447
150,0.521800,0.613825,0.778750,0.774719
175,0.646200,0.594484,0.781250,0.776899
200,0.491800,0.590579,0.778750,0.774286


eval/accuracy,▁▄▇▇▇███
eval/f1,▁▄▇▇████
eval/loss,█▆▄▃▂▁▁▁
eval/runtime,█▁▁▁▁▄▂▁
eval/samples_per_second,▁▇█▇█▄▇█
eval/steps_per_second,▁▇█▇█▄▇█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,██▇██▇▇▇▇▆▇▆▇▅▅▄▆▄▄▃▄▄▂▃▂▂▂▃▂▂▂▃▁▂▁▃▁▁▂▁
train/total_flos,▁


wandb: Agent Starting Run: 1d9md36s with config:
wandb: 	learning_rate: 1e-05
wandb: 	per_device_train_batch_size: 32
wandb: 	weight_decay: 0.25


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ekj4imha with config:
wandb: 	learning_rate: 5e-05
wandb: 	per_device_train_batch_size: 128
wandb: 	weight_decay: 0.25


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  

Step,Training Loss,Validation Loss,Accuracy,F1
25,0.616100,0.528935,0.785000,0.781218
50,0.309300,0.422393,0.831250,0.831444
75,0.216900,0.431455,0.843750,0.845229
100,0.063900,0.401518,0.857500,0.857033
125,0.061700,0.490218,0.851250,0.850211
150,0.056400,0.518607,0.840000,0.840807
175,0.017100,0.492759,0.857500,0.857725
200,0.015800,0.493250,0.856250,0.856615


eval/accuracy,▁▅▇█▇▆██
eval/f1,▁▆▇█▇▆██
eval/loss,█▂▃▁▆▇▆▆
eval/runtime,▄▁▁▅█▅▁▁
eval/samples_per_second,▄█▇▄▁▄██
eval/steps_per_second,▄█▇▄▁▄██
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,██▆▅▅▄▃▃▃▃▂▃▂▂▂▂▂▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
